## Классификация новостей по темам

**Задача:** автоматически определять тематику новости по тексту статьи или заголовка.

### Почему это важно
Автоматическая тематическая классификация позволяет:
- Быстро сортировать и рекомендовать новости пользователям
- Фильтровать контент и строить персональные ленты
- Проводить аналитику больших медиапотоков без ручной разметки
- Повышать точность поиска и таргетинга рекламы

### Данные
- Тексты новостных статей с разметкой по категориям.

### Подход
1. **Разведочный анализ данных** – проверка баланса классов, длины текстов, очистка от мусора.  
2. **Предобработка текста** – лемматизация/стемминг, удаление стоп-слов.  
3. **Векторизация** – TF-IDF. я
4. **Обучение моделей и подбор параметров**
5. **Выбор лучшего решения**

### Модель
- log_reg, xgboost. Используется cuml, вместо sklearn, ввиду большого объёма данных.  
- Метрики: F1-score.

### Результат
- Построена модель, классифицирующая новости по темам с f1=0.86 (log_reg), используя только tf-idf, что можно считать успешным результатом.

### Дальнейшие шаги
- Использование трансформеров


In [ ]:
import pandas as pd
# df = pd.read_csv("/kaggle/input/corpus-of-russian-news-articles-from-lenta/lenta-ru-news.csv")

In [ ]:
# df

In [ ]:
# df.isna().sum()

# 📌 План обработки пропущенных значений

### 1️⃣ **Анализ пропущенных значений**
- **`url`**, **`title`**, **`date`** – нет пропущенных значений, оставляем как есть.
- **`text`** – 5 пропущенных значений. Эти строки будут удалены.
- **`topic`** – 62,002 пропущенных значений. Будем заменять или удалять.
- **`tags`** – 27,219 пропущенных значений. Будем заменять на пустую строку.

### 2️⃣ **Шаги обработки**

#### 🔹 **Обработка `text`**
- Пропущенные значения (5 строк) в **`text`** будут **удалены**, так как без текста статья теряет смысл.

#### 🔹 **Обработка `tags`**
- Пропущенные значения в **`tags`** заменим на **пустую строку** (`""`), чтобы избежать мусора при объединении с текстом.

#### 🔹 **Замена пропущенных `topic` на `tags`**
- Пропущенные значения в **`topic`** (62,002 строки):
  - Мы **не заполняем их случайными значениями**, чтобы не создавать шум.
  - **Если есть `tags`, заполняем `topic` на основе наиболее вероятного `tag`**.
    - Используем **распределение** (какие `tags` чаще всего встречаются с какими `topic`).
    - Если для данного `tag` есть **ясная связь с `topic`** (вероятность >80%), заполняем пропущенный `topic` этим значением.
  - **Если для `tags` нет чёткого соответствия, оставляем `topic` как NaN или "Разное"**.
  
#### 🔹 **Удаление строк с пустыми `topic`**
- Строки, где `topic` остался пустым после замены, будут **удалены**, так как они всё равно не несут полезной информации.

In [ ]:
# import pandas as pd

# # Загружаем датасет (примерные данные)


# # Подсчёт количества (topic, tag)
# tag_topic_counts = df.groupby(['tags', 'topic']).size().reset_index(name='count')

# # Подсчёт общего количества вхождений каждого tag
# tag_counts = df.groupby('tags').size().reset_index(name='total_count')

# # Объединение статистики
# tag_topic_probs = tag_topic_counts.merge(tag_counts, on='tags')
# tag_topic_probs['probability'] = tag_topic_probs['count'] / tag_topic_probs['total_count']

# # Фильтруем "надёжные" замены (например, если вероятность P(topic|tag) > 80%)
# threshold = 0.8
# reliable_replacements = tag_topic_probs[tag_topic_probs['probability'] > threshold]


In [ ]:
# reliable_replacements

In [ ]:
# replacement_dict = reliable_replacements.set_index('tags')['topic'].to_dict()

# # Функция для заполнения пропущенных topic
# def fill_missing_topic(row):
#     if pd.isna(row['topic']) and row['tags'] in replacement_dict:
#         return replacement_dict[row['tags']]
#     return row['topic']

# # Применяем замену
# df['topic'] = df.apply(fill_missing_topic, axis=1)

In [ ]:
# df.isna().sum()

In [ ]:
# # 1️⃣ Заменяем пропущенные `tags` на пустую строку ""
# df['tags'] = df['tags'].fillna('')

# # 2️⃣ Удаляем строки, где `topic` или `text` == NaN
# df = df.dropna(subset=['topic', 'text'])

In [ ]:
# df.isna().sum()


### **Финальный результат**
- **`topic`** с пропусками теперь составляет **46 793** строки, что уменьшилось на **15 209** после замены и удаления. Оставшиеся пропущенные значения удалены.
- **27 219** пропущенных значений в **`tags`** успешно заменены на пустые строки (`""`).
- **5** строк с пропущенным **`text`** были удалены.


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats

# Загрузим данные (здесь заглушка, замените на свой датасет)

def analyze_balance(label_column):


    # Подсчёт количества примеров в каждом классе
    class_counts = label_column.value_counts().sort_index()

    labels = class_counts.index
    values = class_counts.values
    total_samples = len(label_column)
    class_percentages = (class_counts / total_samples) * 100

    # Создание DataFrame для вывода
    class_distribution_df = pd.DataFrame({
        'Количество': values,
        'Процент': class_percentages
    })

    class_distribution_df.reset_index(inplace=True)
    # Форматирование процентов с 6 знаками после запятой
    class_distribution_df['Процент'] = class_distribution_df['Процент'].map(lambda x: f"{x:.3f}")
    print(class_distribution_df)

    print()
    print('-'*20)
    print()

    # Идеальное (равномерное) распределение
    ideal_count = np.mean(values)
    ideal_distribution = np.full_like(values, ideal_count)

    # KL-дивергенция
    kl_div = scipy.stats.entropy(values, ideal_distribution)

    # Вывод KL-дивергенции отдельно
    print('Коэфицент распределение:', kl_div)

    print()
    print('-'*20)
    print()



    # Визуализация
    plt.figure(figsize=(14, 6))
    plt.bar(labels, values, alpha=0.7, label="Фактическое распределение")
    plt.plot(labels, ideal_distribution, color='red', linestyle='dashed', marker='', label="Идеальное распределение")
    plt.xlabel("Классы (topic)")
    plt.ylabel("Количество статей")
    plt.xticks(ticks=np.arange(len(labels)), rotation=60, ha='right')
    plt.title(f"Баланс классов (KL-дивергенция: {kl_div:.4f})")
    plt.legend()
    plt.grid(axis='y', linestyle='--', alpha=0.7)

    # Показ графика
    plt.show()




# analyze_balance(df['topic'])

# Дисбаланс сильный. Некоторые классы представлены чрезвучайно мало (1-3 экземпляра). При этом есть классы, количество экземляров которых более 100000! Разница 100000x. Высокий уровень kl-divergence (0.81) подтверждает это.

# 📌 План обработки дисбаланса данных

### 1️⃣ Объединение редких `topic` в "Разное"
- Если `topic` встречается **менее чем в 0.5%** статей, объединяем его в "Разное".  
- Это помогает корректно обрабатывать **неизвестные темы** и избежать ошибок.  

### 2️⃣ Объединение `tags` и `title` с `text`
- Добавляем `tags` в конец `text` и `title` в начало, сохраняя естественный формат:

[Название статьи]

[Текст статьи]

tags: тег1, тег2, тег3

- `tags` могут помочь модели, но если они бесполезны – она их проигнорирует.  
- `title` содержит ключевую информацию о статье и помогает модели сразу понять её основной контекст.

### 3️⃣ Адаптивный `undersampling`
- Уменьшаем количество примеров у **слишком частых классов** (`>15-20%` датасета).  
- **Формула порога `T%`:**  
`T = (100 / N_classes) * k`  
где `N_classes` – количество классов, а `k` регулирует жёсткость (`k=2` мягко, `k=4` сильно).  
- **Сколько оставить:**  
`new_count_i = max(mean_count, (old_count_i + mean_count) / 2)`

### 4️⃣ Взвешенная лосс-функция (`class_weight`)
- Компенсирует дисбаланс после `undersampling`, чтобы редкие классы не игнорировались.  

### 5️⃣ **Если модель всё равно плохо работает с дисбалансом**  
✅ **Oversampling редких классов:**  
✔ Разбиваем длинные статьи (`>300` слов) на 2-3 части.  
✔ Генерируем новые примеры (GPT) – **только в крайнем случае**.  

### 🚀 Финальный алгоритм  
✔ Объединяем **редкие `topic`** → "Разное".  
✔ **Добавляем `tags` в `text`**.  
✔ **Урезаем слишком частые `topic` (`undersampling`)**.  
✔ **Используем `class_weight`**.  
✔ **Тестируем модель, при необходимости добавляем `oversampling`**.  

🔥 **Гибкий подход для любого датасета!** 🚀




In [ ]:
# Объединяем `title` и `text` в новый столбец `text_with_title`
# df['text_with_title'] = df['title'] + "\n" + df['text']

In [ ]:
# Плюс: можно добавить `tags` в конец (если нужно) или оставить в текущем виде
# df['text'] = df['text_with_title'] + "\ntags: " + df['tags']

In [ ]:
# статьи в датасете не зависят от времени, поэтому date тоже можно удалить :) (ДЛЯ СЕБЯ)
# df = df.drop(columns=['url', 'title', 'tags', 'date', 'text_with_title'])

## Работа с дисбалансом

In [ ]:
# import pandas as pd
# import numpy as np

# # 1️⃣ Объединение редких `topic` в "Разное"
# topic_counts = df['topic'].value_counts()
# threshold = 0.005  # 0.5% от общего количества данных
# min_count = len(df) * threshold

# # # Заменяем редкие темы на "Разное"
# df['topic'] = df['topic'].apply(lambda x: x if topic_counts[x] >= min_count else 'Разное')

In [ ]:
# analyze_balance(df['topic'])

In [ ]:
# N_classes = len(df['topic'].unique())
# k = 2  # Мягкий уровень (можно настроить)
# max_elem_per_class = int((1 / N_classes) * k * len(df['topic']))

In [ ]:
# # 1️⃣ Добавляем индекс каждой строки
# df['index'] = df.index

# # 3️⃣ Обрезаем строки для каждого класса, чтобы их количество не превышало максимальное
# data_balanced = pd.DataFrame()

# for topic, count in topic_counts.items():
#     # Получаем индексы строк для текущего topic
#     topic_data_indices = df[df['topic'] == topic]['index']

#     # Если количество строк больше среднего, обрезаем
#     if count > max_elem_per_class:
#         selected_indices = np.random.choice(topic_data_indices, size=int(max_elem_per_class), replace=False)
#     else:
#         # Если меньше или равно, оставляем все строки
#         selected_indices = topic_data_indices

#     # Добавляем выбранные строки в новый датасет
#     data_balanced = pd.concat([data_balanced, df.loc[df['index'].isin(selected_indices)]])

# # 4️⃣ Удаляем индекс, который использовался для обработки
# data_balanced = data_balanced.drop(columns=['index'])

In [ ]:
# analyze_balance(data_balanced['topic'])

In [ ]:
# data_balanced.to_csv("/kaggle/working/lenta_balanced_df.csv")

# Датасет доступен по ссылке: https://www.kaggle.com/datasets/kehhill/lenta-ru-my-data

In [ ]:
# del df

# 📊 Результаты обработки дисбаланса данных

### 1️⃣ **До обработки**
- **Коэффициент распределения**: 0.812 (значительный дисбаланс классов).
- Пример:
  - `Россия` и `Мир` занимали **21.3%** и **18.2%** от всех данных.
  
### 2️⃣ **После обработки**
- **Коэффициент распределения**: 0.248 (существенное улучшение баланса).
- Пример:
  - `Россия` и `Мир` теперь занимают **15.4%** от всех данных.
- Равномерное распределение классов улучшилось, что поможет модели лучше обучаться.

### 3️⃣ **Что сделано**
- Редкие классы объединены в "Разное", что уменьшило дисбаланс.
- Применён адаптивный `undersampling` для слишком частых классов, что улучшило пропорции.
- Объеденены `title` и `tags` c `text`


In [ ]:
# df = pd.read_csv("/kaggle/input/lenta-ru-my-data/lenta_balanced_df.csv")

In [ ]:
# !pip install pymorphy2

In [ ]:
# import re
# import string
# import pymorphy2
# from nltk.corpus import stopwords
# from nltk.stem import WordNetLemmatizer
# from nltk.corpus import wordnet
# nltk.download("stopwords")
# nltk.download("wordnet")

# # Инициализация лемматизатора
# morph = pymorphy2.MorphAnalyzer()

# # Загружаем стоп-слова
# stop_words = set(stopwords.words("russian"))

# def replace_numbers(text):
#     # Замена дат (например, "9 мая 1945 года" → "_дата_")
#     text = re.sub(r"\b\d{1,2}\s[а-я]+\s\d{4}\b", "_дата_", text)

#     # Замена годов (например, "2023" → "_год_")
#     text = re.sub(r"\b\d{4}\b", "_год_", text)

#     # Замена процентов (например, "50%" → "_процент_")
#     text = re.sub(r"\b\d+%\b", "_процент_", text)
#     text = re.sub(r"\b\d+\sпроцент\w*\b", "_процент_", text)

#     # Замена цен (например, "$100", "100 рублей", "€50" → "_цена_")
#     text = re.sub(r"\b\d+\s?(руб|евро|€|\$|доллар\w*)\b", "_цена_", text)

#     # Замена размеров (например, "100 кг", "2 метра", "150 км" → "_размер_")
#     text = re.sub(r"\b\d+\s?(кг|г|тонн\w*|метр\w*|см|мм|км|л)\b", "_размер_", text)

#     # Остальные числа → "_число_"
#     text = re.sub(r"\b\d+\b", "_число_", text)

#     return text

# def preprocess_text(text):
#     if not isinstance(text, str):
#         return ""

#     text = text.lower()  # Приводим к нижнему регистру

#     # Удаляем HTML-теги
#     text = re.sub(r"<.*?>", "", text)

#     # Заменяем ссылки
#     text = re.sub(r"http\S+|www\S+", "_ссылка_", text)

#     # Заменяем числа по категориям
#     text = replace_numbers(text)

#     # Отделяем важные знаки пунктуации (!, ?, .)
#     text = re.sub(r"([!?.])", r" \1 ", text)

#     # Убираем всю остальную пунктуацию
#     text = text.translate(str.maketrans("", "", string.punctuation.replace("!?.", "")))

#     # Токенизация (по пробелам)
#     words = text.split()

#     # Лемматизация + удаление стоп-слов
#     words = [morph.parse(word)[0].normal_form for word in words if word not in stop_words]

#     return " ".join(words)

# # Применяем к данным
# df["clean_text"] = df["text"].apply(preprocess_text)


In [ ]:
# import pandas as pd
# import multiprocessing as mp
# import re
# import string
# import pymorphy2
# from nltk.corpus import stopwords

# # Лемматизатор pymorphy2
# morph = pymorphy2.MorphAnalyzer()

# # Загружаем стоп-слова
# stop_words = set(stopwords.words("russian"))

In [ ]:
# датасет для проверка скорости очистки текста с multiprocessing и без
# mini_df = df.iloc[:1000]

In [ ]:

# def replace_special_tokens(text):
#     """Обрабатываем числа, ссылки, проценты и т.д."""
#     return re.sub(
#         r"\b\d{1,2}\s[а-я]+\s\d{4}\b", "_дата_",
#         re.sub(r"\b\d{4}\b", "_год_",
#         re.sub(r"\b\d+%\b", "_процент_",
#         re.sub(r"\b\d+\s?(руб|евро|€|\$|доллар\w*)\b", "_цена_",
#         re.sub(r"\b\d+\s?(кг|г|тонн\w*|метр\w*|см|мм|км|л)\b", "_размер_",
#         re.sub(r"\b\d+\b", "_число_",
#         re.sub(r"http\S+|www\S+", "_ссылка_", text)
#         ))))))

# def preprocess_text(text):
#     """Основная функция предобработки текста."""
#     if not isinstance(text, str):
#         return ""

#     text = text.lower()
#     text = replace_special_tokens(text)

#     # Отделяем знаки пунктуации (!, ?, .)
#     text = re.sub(r"([!?.])", r" \1 ", text)
#     text = text.translate(str.maketrans("", "", string.punctuation.replace("!?.", "")))

#     # Токенизация через re.findall
#     words = re.findall(r"\w+|[!?.]", text)

#     # Лемматизация
#     words = [morph.parse(word)[0].normal_form for word in words if word not in stop_words]

#     return " ".join(words)

# def parallel_preprocessing(df, column, num_workers=mp.cpu_count()):
#     """Запускаем предобработку на всех ядрах CPU."""
#     with mp.Pool(num_workers) as pool:
#         df["clean_text"] = pool.map(preprocess_text, df[column])
#     return df




In [ ]:
# import time

# # Засекаем время перед выполнением кода
# start_time_mp = time.time()

# # Применяем к данным
# mini_df = parallel_preprocessing(mini_df, "text")

# # Засекаем время после выполнения
# end_time_mp = time.time()


In [ ]:
# import time

# # Засекаем время перед выполнением кода
# start_time_no_mp = time.time()

# # Применяем к данным
# mini_df["clean_text"] = mini_df["text"].apply(preprocess_text)

# # Засекаем время после выполнения
# end_time_no_mp = time.time()

In [ ]:
# # Выводим время выполнения
# print(f"⏳ Время выполнения с multiproccesing: {end_time_mp - start_time_mp:.2f} секунд")
# # Выводим время выполнения
# print(f"⏳ Время выполнения без multiproccesing: {end_time_no_mp - start_time_no_mp:.2f} секунд")


# Разница больше чем в 2 раза!

In [ ]:
# df = parallel_preprocessing(df, "text")

# для справки: время обработки 650.000 текстов заняло 72 минуты (1 ч 12 минут)

In [ ]:
# df.to_csv("/kaggle/working/df.csv")

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("/kaggle/input/lenta-ru-my-data/clean_texts.csv")

In [ ]:
df.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'text', 'topic', 'clean_text'], dtype='object')

In [ ]:
import gc
import cupy as cp

def clean_all():
    gc.collect()
    cp.get_default_memory_pool().free_all_blocks()

In [ ]:
df = df.drop(columns=['Unnamed: 0.1', 'Unnamed: 0', 'text'], axis=1)

In [ ]:
# df['clean_text'][0]

In [ ]:
import pandas as pd
import cupy as cp  # Работа с массивами на GPU (аналог numpy)
import numpy as np

from cuml.linear_model import LogisticRegression  # GPU-ускоренная логистическая регрессия
import optuna  # Оптимизация гиперпараметров
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split

In [ ]:
import cudf
df_cudf = cudf.DataFrame.from_pandas(df)

# 3️⃣ **Разделение на train/test**
X_train, X_test, y_train, y_test = train_test_split(df_cudf["clean_text"], df_cudf["topic"], test_size=0.2, random_state=2)

from cuml.feature_extraction.text import TfidfVectorizer  # GPU-ускоренный TF-IDF
from cuml.preprocessing import LabelEncoder
# 4️⃣ **TF-IDF на GPU**
vectorizer = TfidfVectorizer(max_features=20000, max_df=0.9, min_df=5)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

X_train_tfidf_cpu = X_train_tfidf.get()  # cupyx.sparse → scipy.sparse
X_test_tfidf_cpu = X_test_tfidf.get()

# Создаём Label Encoder
label_encoder = LabelEncoder()
# Преобразуем `y` в числовой формат
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)

# Перевод cupyx.sparse в scipy.sparse
y_train_cpu = y_train.to_pandas().values
y_test_cpu = y_test.to_pandas().values

In [ ]:
import xgboost as xgb
import scipy.sparse as sp
import cupyx.scipy.sparse as cupyx_sparse
from cuml.linear_model import LogisticRegression
from cuml.svm import SVC
from xgboost import XGBClassifier
import numpy as np
import gc


In [ ]:
clean_all()

In [ ]:
# del df
# del df_cudf

NameError: name 'df' is not defined

## 🎯 Почему считаем веса вручную, а не используем `class_weight="balanced"`?

`class_weight="balanced"` вычисляет веса по формуле:  
**w_c = (общее количество объектов) / (количество классов × объектов данного класса)**.  

Такой метод изменяет веса **линейно**, что плохо работает при сильном дисбалансе: редкие классы могут получать **слишком большие** или **слишком маленькие** веса.  

Мы используем **логарифмическое масштабирование**, чтобы корректировка была более плавной и снижала риск переобучения на редких классах.  





In [ ]:
class_counts = y_train.value_counts().to_pandas()
total_samples = len(y_train)

# Создаём веса (например, логарифмическое масштабирование)
class_weights = {cls: np.log1p(total_samples / count) for cls, count in class_counts.items()}

# Генерируем sample_weight для каждого примера
sample_weights = np.array([class_weights[label] for label in y_train_cpu])

In [ ]:
print(f'Диапозон весов: [{min(class_weights.values()):.2f}, {max(class_weights.values()):.2f}]')

Диапозон весов: [2.01, 4.56]


In [ ]:
import optuna
import cupy as cp
import cudf
from cuml.svm import SVC

def f1_score_gpu(y_true, y_pred, average="macro"):
    """ Реализация F1-score на GPU с cupy """
    y_true = cp.asarray(y_true)  # Преобразуем в cupy массив
    y_pred = cp.asarray(y_pred)

    classes = cp.unique(y_true)  # Находим все классы
    f1_scores = []

    for cls in classes:
        tp = cp.sum((y_pred == cls) & (y_true == cls))
        fp = cp.sum((y_pred == cls) & (y_true != cls))
        fn = cp.sum((y_pred != cls) & (y_true == cls))

        precision = tp / (tp + fp + 1e-9)  # Избегаем деления на 0
        recall = tp / (tp + fn + 1e-9)

        f1 = 2 * (precision * recall) / (precision + recall + 1e-9)
        f1_scores.append(f1)

    f1_scores = cp.array(f1_scores)

    if average == "macro":
        return cp.mean(f1_scores).item()  # Приводим к float
    else:
        return f1_scores  # Возвращаем массив F1 по классам

In [ ]:
batch_size=500

In [ ]:
def train_batch(model, x_train=X_train_tfidf, y_train=y_train, batch_size=batch_size):
    """
    Обучает SVM по батчам, чтобы снизить нагрузку на VRAM.

    Аргументы:
    - x_train: numpy/scipy.sparse - обучающие признаки
    - y_train_cpu: numpy array - метки классов
    - C: float - гиперпараметр SVM
    - batch_size: int - размер батча (по умолчанию 25,000)

    Возвращает:
    - обученную модель SVM
    """
    num_batches = int(np.ceil(x_train.shape[0] / batch_size))
    for batch in range(num_batches):
        start_idx = batch * batch_size
        end_idx = min((batch + 1) * batch_size, x_train.shape[0])

        X_batch = x_train[start_idx:end_idx]
        y_batch = y_train[start_idx:end_idx]

        model.fit(X_batch, y_batch)  # Обучаем на батче

    return model


def predict_batch(model, x_test=X_test_tfidf, batch_size=batch_size):
    """
    Выполняет предсказание SVM по батчам.

    Аргументы:
    - model: обученная модель SVM
    - x_test: numpy/scipy.sparse - тестовые признаки
    - batch_size: int - размер батча

    Возвращает:
    - y_pred: numpy array - объединённые предсказания модели
    """

    num_batches = int(np.ceil(x_test.shape[0] / batch_size))
    y_pred = []

    for batch in range(num_batches):
        start_idx = batch * batch_size
        end_idx = min((batch + 1) * batch_size, x_test.shape[0])

        X_batch = x_test[start_idx:end_idx]
        y_batch_pred = model.predict(X_batch)  # Предсказание для батча
        y_pred.append(y_batch_pred)

    return np.concatenate(y_pred)



In [ ]:
# import cudf
# df_cudf = cudf.DataFrame.from_pandas(df)

# # 3️⃣ **Разделение на train/test**
# X_train, X_test, y_train, y_test = train_test_split(df_cudf["clean_text"], df_cudf["topic"], test_size=0.2, random_state=2)

# from cuml.feature_extraction.text import TfidfVectorizer  # GPU-ускоренный TF-IDF
# from cuml.preprocessing import LabelEncoder
# # 4️⃣ **TF-IDF на GPU**
# vectorizer = TfidfVectorizer(max_features=20000, max_df=0.9, min_df=5)
# X_train_tfidf = vectorizer.fit_transform(X_train)
# X_test_tfidf = vectorizer.transform(X_test)

# X_train_tfidf_cpu = X_train_tfidf.get()  # cupyx.sparse → scipy.sparse
# X_test_tfidf_cpu = X_test_tfidf.get()

# # Создаём Label Encoder
# label_encoder = LabelEncoder()
# # Преобразуем `y` в числовой формат
# y_train = label_encoder.fit_transform(y_train)
# y_test = label_encoder.transform(y_test)

# # Перевод cupyx.sparse в scipy.sparse
# y_train_cpu = y_train.to_pandas().values
# y_test_cpu = y_test.to_pandas().values

In [ ]:
import xgboost as xgb
import numpy as np

def train_batch_xgb(params, x_train_cpu=X_train_tfidf_cpu, y_train_cpu=y_train_cpu, sample_weights=sample_weights, batch_size=batch_size, num_boost_round=50):
    """
    Обучает XGBoost по батчам, чтобы снизить нагрузку на VRAM.

    Аргументы:
    - X_train_cpu: numpy/scipy.sparse - обучающие признаки
    - y_train_cpu: numpy array - метки классов
    - params: dict - параметры XGBoost
    - batch_size: int - размер батча (по умолчанию 25,000)
    - num_boost_round: int - число деревьев в каждом батче

    Возвращает:
    - model: обученная модель XGBoost
    """

    num_batches = int(np.ceil(x_train_cpu.shape[0] / batch_size))  # Количество батчей
    model = None

    for batch in range(num_batches):
        start_idx = batch * batch_size
        end_idx = min((batch + 1) * batch_size, x_train_cpu.shape[0])

        X_batch = x_train_cpu[start_idx:end_idx]
        y_batch = y_train_cpu[start_idx:end_idx]
        batch_sample_weights = sample_weights[start_idx:end_idx]
        dtrain = xgb.DMatrix(X_batch, label=y_batch, weight=batch_sample_weights)

        if model is None:
            # Первое обучение (инициализация модели)
            model = xgb.train(params, dtrain, num_boost_round=num_boost_round)
        else:
            # Обновление модели на следующем батче
            model = xgb.train(params, dtrain, num_boost_round=num_boost_round, xgb_model=model)

        # print(f"🔄 Батч {batch+1}/{num_batches} обучен.")

    return model


In [ ]:
import xgboost as xgb
import numpy as np

def predict_batch_xgb(model, x_test_cpu=X_test_tfidf_cpu, batch_size=batch_size):
    """
    Выполняет предсказание модели XGBoost по батчам.

    Аргументы:
    - model: обученная модель XGBoost
    - x_test_cpu: numpy/scipy.sparse - тестовые признаки
    - batch_size: int - размер батча (по умолчанию 25,000)

    Возвращает:
    - y_pred: numpy array - объединённые предсказания модели
    """

    num_batches = int(np.ceil(x_test_cpu.shape[0] / batch_size))  # Количество батчей
    y_pred = []  # Хранилище предсказаний

    for batch in range(num_batches):
        start_idx = batch * batch_size
        end_idx = min((batch + 1) * batch_size, x_test_cpu.shape[0])

        X_batch = x_test_cpu[start_idx:end_idx]
        dtest_batch = xgb.DMatrix(X_batch)  # Преобразуем в DMatrix

        y_batch_pred = model.predict(dtest_batch)  # Предсказание для батча
        y_pred.append(y_batch_pred)

        # print(f"🔄 Батч {batch+1}/{num_batches} предсказан.")

    return np.concatenate(y_pred)  # Объединяем предсказания

# 🚀 Оптимизация гиперпараметров с Optuna и GPU-ускорением

## 📌 Описание
Этот модуль выполняет автоматический подбор гиперпараметров для трёх GPU-ускоренных моделей:
- **Logistic Regression** (из библиотеки cuML, вместо sklearn)
- **SVM** (из библиотеки cuML, вместо sklearn)
- **XGBoost** (с оптимизацией под GPU)

Оптимизация построена с учётом производительности:  
- **Для XGBoost используется `max_bin=128` на этапе Optuna** (ускорение подбора).  
  Если XGBoost окажется лучшей моделью, то финальное обучение выполняется с **`max_bin=512-1024`** для повышения точности.  
- **XGBoost обучается батчами (по 25,000 записей), чтобы снизить нагрузку на VRAM** и избежать Out of Memory (OOM).  


In [ ]:
clean_all()

In [ ]:
import optuna
from cuml.linear_model import LogisticRegression as CuLogReg
import time
def objective(trial):
    C = trial.suggest_float("C", 1e-3, 1e3)
    penalty = trial.suggest_categorical("penalty", ["l1", "l2"])


    model = CuLogReg(C=C, penalty=penalty, max_iter=2500, class_weight="balanced")
    model.fit(X_train_tfidf, y_train)
    y_pred = model.predict(X_test_tfidf)
    score = f1_score_gpu(y_test, y_pred, average="macro")


    del model  # Удаляем объект модели
    gc.collect()  # Очищаем память

    return score

start_time= time.time()
study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(), pruner=optuna.pruners.HyperbandPruner())
study.optimize(objective, n_trials=5)
end_time = time.time()
print(study.best_params)
print(f'Затраченное время на поиск параметров {end_time-start_time:.2f}')

[I 2025-02-27 04:22:54,641] A new study created in memory with name: no-name-58c2e3f0-5d0d-4d70-bc50-449803d314b7
[I 2025-02-27 04:23:42,002] Trial 0 finished with value: 0.842590440166971 and parameters: {'C': 493.2229750563484, 'penalty': 'l2'}. Best is trial 0 with value: 0.842590440166971.


[2025-02-27 04:23:41.837] [CUML] [warning] L-BFGS: max iterations reached
[2025-02-27 04:23:41.838] [CUML] [warning] Maximum iterations reached before solver is converged. To increase model accuracy you can increase the number of iterations (max_iter) or improve the scaling of the input data.


[I 2025-02-27 04:24:28,215] Trial 1 finished with value: 0.7876429326294473 and parameters: {'C': 806.5071135838357, 'penalty': 'l1'}. Best is trial 0 with value: 0.842590440166971.


[2025-02-27 04:24:28.049] [CUML] [warning] QWL-QN: max iterations reached
[2025-02-27 04:24:28.050] [CUML] [warning] Maximum iterations reached before solver is converged. To increase model accuracy you can increase the number of iterations (max_iter) or improve the scaling of the input data.


[I 2025-02-27 04:25:15,682] Trial 2 finished with value: 0.7880714586076606 and parameters: {'C': 857.5529782049359, 'penalty': 'l1'}. Best is trial 0 with value: 0.842590440166971.


...

[2025-02-27 04:34:32.578] [CUML] [warning] L-BFGS: max iterations reached
[2025-02-27 04:34:32.580] [CUML] [warning] Maximum iterations reached before solver is converged. To increase model accuracy you can increase the number of iterations (max_iter) or improve the scaling of the input data.


[I 2025-02-27 04:35:17,964] Trial 17 finished with value: 0.7919423372807787 and parameters: {'C': 398.656509655662, 'penalty': 'l1'}. Best is trial 4 with value: 0.8612553647060918.
[I 2025-02-27 04:36:05,379] Trial 18 finished with value: 0.7871455291563072 and parameters: {'C': 957.0255716787765, 'penalty': 'l1'}. Best is trial 4 with value: 0.8612553647060918.


[2025-02-27 04:36:05.219] [CUML] [warning] QWL-QN: max iterations reached
[2025-02-27 04:36:05.220] [CUML] [warning] Maximum iterations reached before solver is converged. To increase model accuracy you can increase the number of iterations (max_iter) or improve the scaling of the input data.


[I 2025-02-27 04:36:52,071] Trial 19 finished with value: 0.8407171672085081 and parameters: {'C': 172.2703905965826, 'penalty': 'l2'}. Best is trial 4 with value: 0.8612553647060918.


[2025-02-27 04:36:51.918] [CUML] [warning] L-BFGS: max iterations reached
[2025-02-27 04:36:51.920] [CUML] [warning] Maximum iterations reached before solver is converged. To increase model accuracy you can increase the number of iterations (max_iter) or improve the scaling of the input data.
{'C': 7.455048798372313, 'penalty': 'l2'}
Затраченное время на поиск параметров 837.43


{'C': 7.455048798372313, 'penalty': 'l2'}
Затраченное время на поиск параметров 837.43

In [ ]:
if best_params["model"] == "LogisticRegression":
    final_model = LogisticRegression(C=best_params["C"], max_iter=1000, class_weight="balanced")
elif best_params["model"] == "XGBoost":
    final_model = XGBClassifier(n_estimators=best_params["n_estimators"], max_depth=best_params["max_depth"], learning_rate=best_params["learning_rate"],
                                tree_method="gpu_hist", use_label_encoder=False, eval_metric="mlogloss",
                                scale_pos_weight=class_weights.to_dict())

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
final_model.fit(X_train_tfidf, y_train)
y_pred = final_model.predict(X_test_tfidf)

# 7️⃣ **Оценка модели**
print("📊 Итоговые метрики модели:")
print(classification_report(y_test, y_pred))

In [ ]:

# mini_cpu_x_train = X_train_tfidf_cpu[:1000]
# mini_cpu_y_train = y_train_cpu[:1000]
# mini_cpu_x_test = X_test_tfidf_cpu[:1000]
# mini_cpu_y_test = y_test_cpu[:1000]

In [ ]:
import xgboost as xgb
import optuna
import torch
import time
from sklearn.metrics import f1_score

def objective(trial):
    params = {
        "num_class": 14,
        "tree_method": "hist",  # ❗ Используем "hist" + "device=cuda"
        "device": "cuda",
        "max_bin": 128,
        "learning_rate": trial.suggest_float("learning_rate", 0.1, 0.3, log=True),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "subsample": 1.0,   # (trial.suggest_float("subsample", 0.5, 1.0, log=True))
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0, log=True),
        "gamma": trial.suggest_float("gamma", 0.25, 1.0, log=True),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 1.0, log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 1.0, log=True)
    }
    torch.cuda.empty_cache()

    model = train_batch_xgb(params, batch_size=12500)
    y_pred = predict_batch_xgb(model, batch_size=12500)

    score = f1_score(y_test_cpu, y_pred, average="macro")

    # model = train_batch_xgb(params, x_train_cpu=mini_cpu_x_train, y_train_cpu=mini_cpu_y_train, batch_size=1000)
    # y_pred = predict_batch_xgb(model, x_test_cpu=mini_cpu_x_test, batch_size=1000)


    del model  # Удаляем объект модели
    gc.collect()  # Очищаем память

    return score


start_time= time.time()
study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(), pruner=optuna.pruners.HyperbandPruner())
study.optimize(objective, n_trials=5)
end_time = time.time()
print(study.best_params)
print(f'Затраченное время на поиск параметров {end_time-start_time:.2f}')

[I 2025-02-27 06:51:46,706] A new study created in memory with name: no-name-c4f2be85-c2c2-478a-9b1c-3ff56699b4a0
[I 2025-02-27 06:52:05,371] Trial 0 finished with value: 0.5832141820033293 and parameters: {'learning_rate': 0.017444069879670766, 'max_depth': 8, 'subsample': 0.7894486974787484, 'colsample_bytree': 0.815915190703879, 'gamma': 9.260583049801996e-05, 'reg_alpha': 1.1052647888167963e-07, 'reg_lambda': 5.245151773884428e-06}. Best is trial 0 with value: 0.5832141820033293.
[I 2025-02-27 06:52:09,302] Trial 1 finished with value: 0.5556913951516981 and parameters: {'learning_rate': 0.05460351713442529, 'max_depth': 3, 'subsample': 0.5445247734632983, 'colsample_bytree': 0.7263891783488546, 'gamma': 0.0009082168286551289, 'reg_alpha': 0.08041307165707874, 'reg_lambda': 5.524661025940831e-07}. Best is trial 0 with value: 0.5832141820033293.
[I 2025-02-27 06:52:21,221] Trial 2 finished with value: 0.5674784470740049 and parameters: {'learning_rate': 0.19855023652049011, 'max_dep

KeyboardInterrupt: 

In [ ]:
best_params = study.best_params_
print(f"🏆 Лучшие параметры: {best_params}")

In [ ]:
if best_params["model"] == "LogisticRegression":
    final_model = LogisticRegression(C=best_params["C"], max_iter=1000, class_weight="balanced")
elif best_params["model"] == "XGBoost":
    final_model = XGBClassifier(n_estimators=best_params["n_estimators"], max_depth=best_params["max_depth"], learning_rate=best_params["learning_rate"],
                                tree_method="gpu_hist", use_label_encoder=False, eval_metric="mlogloss",
                                scale_pos_weight=class_weights.to_dict())

In [ ]:
final_model.fit(X_train_tfidf, y_train)
y_pred = final_model.predict(X_test_tfidf)

# 7️⃣ **Оценка модели**
print("📊 Итоговые метрики модели:")
print(classification_report(y_test, y_pred))